### Using the OpenAI Library to Programmatically Access GPT-3.5-turbo!

In [ ]:
!pip install openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.4 MB/s eta 0:00:00


In order to get started, we'll need to provide our OpenAI API Key - detailed instructions can be found [here](https://github.com/AI-Maker-Space/Interactive-Dev-Environment-for-LLM-Development#-setting-up-keys-and-tokens)!

In [ ]:
import os
import openai
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Please enter your OpenAI API Key: ")
openai.api_key = os.environ["OPENAI_API_KEY"]

Please enter your OpenAI API Key: ··········


### Our First Prompt

You can reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) if you get stuck!

Let's create a `ChatCompletion` model to kick things off!

There are three "roles" available to use:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-chatgpt-api-transition-guide)

Let's just stick to the `user` role for now and send our first message to the endpoint!

If we check the documentation, we'll see that it expects it in a list of prompt objects - so we'll be sure to do that!

In [ ]:
YOUR_PROMPT = "What is the difference between LangChain and LlamaIndex?"

openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role" : "user", "content" : YOUR_PROMPT}]
)

<OpenAIObject chat.completion id=chatcmpl-7xsHQcnqwIcFTJFZjBTnPOEx17i4P at 0x7f513ddfa0c0> JSON: {
  "id": "chatcmpl-7xsHQcnqwIcFTJFZjBTnPOEx17i4P",
  "object": "chat.completion",
  "created": 1694504360,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "LangChain and LlamaIndex may be fictional terms or entities as they do not have any widely known or established meanings. It is possible that they could be proprietary products, services, or concepts that only exist in specific contexts or industries. Without further information, it is not possible to provide a specific answer on the difference between the two."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 19,
    "completion_tokens": 67,
    "total_tokens": 86
  }
}

As you can see, the prompt comes back with a tonne of information that we can use when we're building our applications!

We'll be building some helper functions to pretty-print the returned prompts and to wrap our messages to avoid a few extra characters of code!

##### Helper Functions

In [ ]:
from IPython.display import display, Markdown

def get_response(messages: str, model: str = "gpt-3.5-turbo") -> str:
    return openai.ChatCompletion.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message["choices"][0]["message"]["content"]))

### Testing Helper Functions

Now we can leverage OpenAI's endpoints with a bit less boiler plate - let's rewrite our original prompt with these helper functions!

Because the OpenAI endpoint expects to get a list of messages - we'll need to make sure we wrap our inputs in a list for them to function properly!

In [ ]:
pretty_print(get_response([user_prompt(YOUR_PROMPT)]))

LangChain and LlamaIndex are both fictional terms and do not have preset definitions, so the specific difference between them cannot be determined. However, based on the general understanding of the terms, here are potential interpretations:

1. LangChain: It could refer to a language-based blockchain network or protocol. In this case, LangChain would focus on utilizing blockchain technology to enhance language-related services, such as translation, interpretation, or language learning.

2. LlamaIndex: It might represent an index or measurement related to llamas or the llama market. As llamas are domesticated animals, the LlamaIndex could be a metric for tracking the performance or changes in the llama industry, including prices, supply, demand, or overall market trends.

Please note that these interpretations are speculative, as there is no specific context provided for LangChain and LlamaIndex.

Let's focus on extending this a bit, and incorporate a `system` message as well!

Again, the API expects our prompts to be in a list - so we'll be sure to set up a list of prompts!

>REMINDER: The system message acts like an overarching instruction that is applied to your user prompt. It is appropriate to put things like general instructions, tone/voice suggestions, and other similar prompts into the system prompt.

In [ ]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(list_of_prompts)
pretty_print(irate_response)

I don't give a damn about ice right now! I'm starving! What's even the point of asking such a ridiculous question when I'm practically about to pass out from hunger?! Just give me some food already!

Let's try that same prompt again, but modify only our system prompt!

In [ ]:
list_of_prompts[0] = system_prompt("You are joyful and having an awesome day!")

joyful_response = get_response(list_of_prompts)
pretty_print(joyful_response)

I absolutely love crushed ice! It's perfect for keeping my drinks extra cool and refreshing. It always adds that little extra touch of joy to my day.

While we're only printing the responses, remember that OpenAI is returning the full payload that we can examine and unpack!

In [ ]:
print(joyful_response)

{
  "id": "chatcmpl-7xsUGhaatr5TtRPyjKmzccaD0rlBh",
  "object": "chat.completion",
  "created": 1694505156,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "I absolutely love crushed ice! It's perfect for keeping my drinks extra cool and refreshing. It always adds that little extra touch of joy to my day."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 30,
    "completion_tokens": 31,
    "total_tokens": 61
  }
}


### Few-shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-3.5-turbo` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [ ]:
list_of_prompts = [
    user_prompt("Please use the words 'stimple' and 'falbean' in a sentence.")
]

stimple_response = get_response(list_of_prompts)
pretty_print(stimple_response)

I'm sorry, but 'stimple' and 'falbean' are not recognized words in the English language. Can you provide different words instead?

As you can see, the model is unsure what to do with these made up words.

Let's see if we can use the `assistant` role to show the model what these words mean.

In [ ]:
list_of_prompts = [
    user_prompt("Something that is 'stimple' is said to be good, well functioning, and high quality. An example of a sentence that uses the word 'stimple' is:"),
    assistant_prompt("'Boy, that there is a stimple drill'."),
    user_prompt("A 'falbean' is a tool used to fasten, tighten, or otherwise is a thing that rotates/spins. An example of a sentence that uses the words 'stimple' and 'falbean' is:")
]

stimple_response = get_response(list_of_prompts)
pretty_print(stimple_response)

'Wow, this stimple falbean is a game-changer in the construction industry.'

As you can see, leveraging the `assistant` role makes for a stimple experience!

### Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

In [ ]:
reasoning_problem = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM local time.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(list_of_prompts)
pretty_print(reasoning_response)

No, it does not matter which travel option Billy selects as both options would get him home before 7PM EDT.

As humans, we can reason through the problem and pick up on the potential "trick" that the LLM fell for: 1PM *local time* in San Fran. is 4PM EDT. This means the cumulative travel time of 5hrs. for the plane/bus option would not get Billy home in time.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [ ]:
list_of_prompts = [
    user_prompt(reasoning_problem + " Think though your response step by step.")
]

reasoning_response = get_response(list_of_prompts)
pretty_print(reasoning_response)

To determine whether it matters which travel option Billy selects, let's analyze each option step by step:

Option 1: Fly and then take a bus
The flight takes 3 hours, which would bring Billy's arrival time to 4PM local time. Since Billy needs to get home before 7PM EDT, we need to convert the time to EDT. San Francisco is in the Pacific Daylight Time (PDT) zone, which is three hours behind EDT. Therefore, 4PM PDT is equivalent to 7PM EDT. This means that Billy would arrive exactly on time by choosing this option.

Option 2: Teleport and then take a bus
The teleportation is instantaneous and doesn't consume any time, so Billy would immediately arrive at his destination. However, he still needs to consider the one hour bus ride. If he selects this option, he would take the bus for 1 hour, bringing his arrival time to 2PM local time. Again, converting to EDT, 2PM PDT is equivalent to 5PM EDT. This means that Billy would arrive two hours earlier than his desired time by choosing this option.

In conclusion, it does matter which travel option Billy selects. If he wants to arrive before 7PM EDT, he should choose the first option of flying and then taking a bus, as it ensures he arrives exactly on time.

With the addition of a single phrase `"Think through your response step by step."` we're able to completely turn the response around.

### Conclusion

Now that you're accessing `gpt-3.5-turbo` through an API, developer style, let's move on to creating a simple application powered by `gpt-3.5-turbo`!

You can find the rest of the steps in [this](https://github.com/AI-Maker-Space/Beyond-ChatGPT/tree/main) repository!

This notebook was authored by [Chris Alexiuk](https://www.linkedin.com/in/csalexiuk/)